<a href="https://colab.research.google.com/github/samartha007/Psoriasis-detection-using-transfer-learning/blob/New-random-sampling-method/Alternative_sampling_approach_Mobile_net_parameter_tuweking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten,Dropout
from tensorflow.keras.optimizers import Adam, SGD
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix,roc_curve,auc


from keras.applications.mobilenet import MobileNet
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
#### Modeling
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,BatchNormalization,Dropout
from tensorflow.keras.optimizers import Adam,SGD
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications import EfficientNetB0
from keras.applications.mobilenet import MobileNet
from keras.applications.vgg19 import VGG19
from tensorflow.keras.applications import ResNet50
from sklearn.model_selection import cross_val_score,cross_val_predict, StratifiedKFold
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn import metrics
from sklearn.metrics import roc_curve, auc



In [2]:
from google.colab import drive

drive.mount('/content/gdrive/', force_remount=True)

Mounted at /content/gdrive/


In [3]:
import os
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import preprocess_input

# Path to the directories containing the class data
class_1_directory = '/content/gdrive/MyDrive/all patch data/all patch data/disease_with_online_data'
class_2_directory = '/content/gdrive/MyDrive/all patch data/all patch data/normal'

# Get a list of image filenames for each class
class_1_filenames = os.listdir(class_1_directory)
class_2_filenames = os.listdir(class_2_directory)

# Create empty lists to store the data and labels
data = []
labels = []

# Loop over each image in class 1
for image_filename in class_1_filenames:
    # Read the image
    image_path = os.path.join(class_1_directory, image_filename)
    image = Image.open(image_path)

    # Preprocess the image for VGG16
    image = image.resize((224, 224))
    # image = image.convert('RGB')
    image = np.array(image)
    # image = preprocess_input(image)

    # Add the preprocessed image and corresponding label to the lists
    data.append(image)
    labels.append(0)  # Class 1 is labeled as 0

# Loop over each image in class 2
for image_filename in class_2_filenames:
    # Read the image
    image_path = os.path.join(class_2_directory, image_filename)
    image = Image.open(image_path)

    # Preprocess the image for VGG16
    image = image.resize((224, 224))
    # image = image.convert('RGB')
    image = np.array(image)
    # image = preprocess_input(image)

    # Add the preprocessed image and corresponding label to the lists
    data.append(image)
    labels.append(1)  # Class 2 is labeled as 1

# Convert the lists to NumPy arrays
data = np.array(data)
labels = np.array(labels)
# # Normalize the image data
data = data / 255.0
# Split the data into training and testing sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(data, labels, test_size=0.2, random_state=42)


## For Mobile Net application using the Newly mentioned method

In [6]:
#### SIR Newly Mentioned Logic of sampling
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
sns=[]
spc=[]
AUC=[]
validation_loss=[]
validation_accuracy=[]
# Assuming you have your feature data in 'X' and labels in 'y'
# Define your model as 'model'
# Define the number of epochs as 'num_epochs'
def sir_method(base,height,width):
# Get the total number of samples
  num_samples = len(labels)
  num_epochs=10
  train_ratio = 0.8
  # Convert train_ratio to an absolute number of samples
  train_size = int(train_ratio * len(data))
  for epoch in range(num_epochs):
# Randomly split the data into training and validation sets for this epoch
      X_train, X_val, y_train, y_val = train_test_split(data, labels, train_size=train_size, random_state=epoch)
      base_model = base(weights='imagenet', include_top=False, input_shape=(height, width, 3))

        # Set up the model architecture
      model = Sequential()
      model.add(base_model)
      model.add(Flatten())
      model.add(Dense(height, activation='relu'))
      model.add(Dropout(0.5))
      model.add(Dense(1, activation='sigmoid'))
      base_model.trainable = False

      # Compile the model
      model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
      # Train the model on the training set
      model.fit(X_train, y_train, epochs=1, batch_size=50, verbose=0)
      # Evaluate the model on the validation set
      val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
      validation_loss.append(val_loss)
      validation_accuracy.append(val_acc)
      # Make predictions on the validation set
      y_pred = model.predict(X_val)

      # Calculate sensitivity, specificity, and AUC
      cm = confusion_matrix(y_val, np.round(y_pred))
      tn, fp, fn, tp = cm.ravel()
      sensitivity = tp / (tp + fn)
      specificity = tn / (tn + fp)
      auc = roc_auc_score(y_val, y_pred)
      sns.append(sensitivity)
      spc.append(specificity)
      AUC.append(auc)

      # Print the evaluation metrics
      # print(f"Epoch {epoch+1} - Sensitivity: {sensitivity:.4f} - Specificity: {specificity:.4f} - AUC: {auc:.4f}")
  return sns,spc,AUC,validation_loss,validation_accuracy


In [7]:
mobile_net_sns,mobile_net_spc,mobile_net_auc,mobile_net_val_loss,mobile_net_val_acc=sir_method(MobileNet,224,224)

3/3 [==============================] - 1s 30ms/step


3/3 [==============================] - 1s 32ms/step


3/3 [==============================] - 1s 41ms/step


In [8]:
print(f'Mean sensitivity of MobileNet after 10 epochs {np.mean(mobile_net_sns)}')
print(f'Mean specificity after 10 epochs {np.mean(mobile_net_spc)}')
print(f'Mean AUC after 10 epochs {np.mean(mobile_net_auc)}')

Mean sensitivity of MobileNet after 10 epochs 0.9285981986949728
Mean specificity after 10 epochs 0.9085221691682797
Mean AUC after 10 epochs 0.9845283197541465


### For Resnet-50 application

In [9]:
res_net_sns,res_net_spc,res_net_auc,res_net_val_loss,res_net_val_acc=sir_method(ResNet50,224,224)

3/3 [==============================] - 1s 83ms/step


In [10]:
print(f'Mean sensitivity of VGG16 after 10 epochs {np.mean(res_net_sns)}')
print(f'Mean specificity of VGG16 after 10 epochs {np.mean(res_net_spc)}')
print(f'Mean AUC of VGG16 after 10 epochs {np.mean(res_net_auc)}')

Mean sensitivity of VGG16 after 10 epochs 0.6142990993474864
Mean specificity of VGG16 after 10 epochs 0.8066185407244909
Mean AUC of VGG16 after 10 epochs 0.9479971040787876


## For VGG 16 and 19 application

In [11]:
vgg16_sns,vgg16_spc,vgg16_auc,vgg16_val_loss,vgg16_val_acc=sir_method(VGG16,224,224)

3/3 [==============================] - 0s 122ms/step


In [12]:
print(f'Mean sensitivity of VGG16 after 10 epochs {np.mean(vgg16_sns)}')
print(f'Mean specificity of VGG16 after 10 epochs {np.mean(vgg16_spc)}')
print(f'Mean AUC of VGG16 after 10 epochs {np.mean(vgg16_auc)}')

Mean sensitivity of VGG16 after 10 epochs 0.6754573327906662
Mean specificity of VGG16 after 10 epochs 0.8094126366029606
Mean AUC of VGG16 after 10 epochs 0.936981443545062


In [13]:
vgg19_sns,vgg19_spc,vgg19_auc,vgg19_val_loss,vgg19_val_acc=sir_method(VGG19,224,224)

3/3 [==============================] - 1s 148ms/step


In [14]:
print(f'Mean sensitivity of VGG19 after 10 epochs {np.mean(vgg19_sns)}')
print(f'Mean specificity of VGG19 after 10 epochs {np.mean(vgg19_spc)}')
print(f'Mean AUC of VGG19 after 10 epochs {np.mean(vgg19_auc)}')

Mean sensitivity of VGG19 after 10 epochs 0.6563112929483897
Mean specificity of VGG19 after 10 epochs 0.8305280346902351
Mean AUC of VGG19 after 10 epochs 0.9377533541148553


## Fine Tuning of Mobile Net

In [15]:
#### SIR Newly Mentioned Logic of sampling
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, confusion_matrix
sns=[]
spc=[]
AUC=[]
validation_loss=[]
validation_accuracy=[]
# Assuming you have your feature data in 'X' and labels in 'y'
# Define your model as 'model'
# Define the number of epochs as 'num_epochs'
def sir_method_fine_tuned_model(base,height,width):
# Get the total number of samples
  num_samples = len(labels)
  num_epochs=10
  train_ratio = 0.8
  # Convert train_ratio to an absolute number of samples
  train_size = int(train_ratio * len(data))
  for epoch in range(num_epochs):
      # Shuffle the data and labels
      X_train, X_val, y_train, y_val = train_test_split(data, labels, train_size=train_size, random_state=epoch)
      base_model = base(weights='imagenet', include_top=False, input_shape=(height, width, 3))

        # Set up the model architecture
      model = Sequential()
      model.add(base_model)
      model.add(Flatten())
      model.add(Dense(height, activation='relu'))
      model.add(Dropout(0.5))
      model.add(Dense(1, activation='sigmoid'))
      base_model.trainable = False

      # Compile the model
      model.compile(optimizer=Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])
      # Train the model on the training set
      model.fit(X_train, y_train, epochs=1, batch_size=50, verbose=0)
      # Evaluate the model on the validation set
      val_loss, val_acc = model.evaluate(X_val, y_val, verbose=0)
      validation_loss.append(val_loss)
      validation_accuracy.append(val_acc)
      # Make predictions on the validation set
      y_pred = model.predict(X_val)

      # Calculate sensitivity, specificity, and AUC
      cm = confusion_matrix(y_val, np.round(y_pred))
      tn, fp, fn, tp = cm.ravel()
      sensitivity = tp / (tp + fn)
      specificity = tn / (tn + fp)
      auc = roc_auc_score(y_val, y_pred)
      sns.append(sensitivity)
      spc.append(specificity)
      AUC.append(auc)

      # Print the evaluation metrics
      # print(f"Epoch {epoch+1} - Sensitivity: {sensitivity:.4f} - Specificity: {specificity:.4f} - AUC: {auc:.4f}")
  return sns,spc,AUC,validation_loss,validation_accuracy


In [16]:
fine_tuned_sns,fine_tuned_spc,fine_tuned_auc,fine_tuned_val_loss,fine_tuned_val_acc=sir_method_fine_tuned_model(MobileNet,224,224)

3/3 [==============================] - 1s 31ms/step


In [17]:
print(f'Mean sensitivity of fine tuned MobileNet after 10 epochs {np.mean(fine_tuned_sns)}')
print(f'Standard deviation sensitivity of fine tuned MobileNet after 10 epochs {np.var(fine_tuned_sns)}')
print(f'Mean specificity of fine tuned MobileNet after 10 epochs {np.mean(fine_tuned_spc)}')
print(f'Standard deviation specificity of fine tuned MobileNet after 10 epochs {np.var(fine_tuned_spc)}')
print(f'Mean AUC of fine tuned MobileNet after 10 epochs {np.mean(fine_tuned_auc)}')
print(f'Standard deviation AUC of fine tuned MobileNet after 10 epochs {np.var(fine_tuned_auc)}')

Mean sensitivity of fine tuned MobileNet after 10 epochs 0.9602826027019576
Standard deviation sensitivity of fine tuned MobileNet after 10 epochs 0.0025535391728199693
Mean specificity of fine tuned MobileNet after 10 epochs 0.9136180637818516
Standard deviation specificity of fine tuned MobileNet after 10 epochs 0.001818929924961303
Mean AUC of fine tuned MobileNet after 10 epochs 0.9883012239606719
Standard deviation AUC of fine tuned MobileNet after 10 epochs 0.00013964981141312413


In [18]:
np.mean(fine_tuned_val_acc)

0.9324324309825898